## Часть 1
Напишите скрипт, который скачивает все данные с выборов губернатора СПб 2019г для всех избирательных участков. 

Входная точка [по ссылке](http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [1]:
import lxml.html
import lxml.etree
import requests

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222'
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")

In [4]:
tik = soup.findAll('a',href=True, style="text-decoration: none")

In [8]:
df = pd.DataFrame()

for i in range(len(tik) - 1):
    
    url_i = 'http://notelections.online' + tik[i].get('href')
    yik = requests.get(url_i)
    soup_i = BeautifulSoup(yik.text, "lxml")
    
    table = soup_i.find_all('table')[2]
    df_i = pd.read_html(str(table))
    
    df_i = df_i[6].T
    df_i = df_i.drop(columns=[12])
    df_i = df_i.rename(columns={0: "yik", 13: "12", 14: "13", 15: "14"})
    
    df_i['yik'] = [int(j.split()[1][1:]) for j in df_i['yik'].tolist()]
    df_i['12'] = [int(j.split()[0]) for j in df_i['12'].tolist()]
    df_i['13'] = [int(j.split()[0]) for j in df_i['13'].tolist()]
    df_i['14'] = [int(j.split()[0]) for j in df_i['14'].tolist()]
    df_i['tik'] = i + 1
    
    cols = df_i.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df_i = df_i[cols]
    df = pd.concat([df, df_i], ignore_index=True)

In [9]:
df

,tik,yik,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,1,1803,1200,587,11,602,11,587,23,575,0,0,110,345,120
1,1,2,1466,1100,433,14,653,14,430,19,425,0,0,53,326,46
2,1,3,2092,1600,576,22,1002,22,576,15,583,0,0,155,332,96
3,1,4,1056,1000,318,4,678,4,318,13,309,0,0,67,171,71
4,1,5,1827,1400,495,8,897,8,493,16,485,0,0,137,266,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,30,2350,284,300,166,53,81,53,166,0,219,0,0,21,184,14
2017,30,2351,132,150,86,27,37,27,86,2,111,0,0,7,91,13
2018,30,2352,320,350,217,23,110,23,217,4,236,0,0,20,193,23
2019,30,2355,485,500,335,12,153,12,335,2,345,0,0,23,290,32


In [10]:
df.to_csv(r'C:\Users\user\Desktop\pm\Mishenin\TASK\dataframe.csv', index=False)